In [2]:
import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
import support as S


C:\Users\Acs\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#Root directory of the project
ROOT_DIR = os.path.abspath(".")
TRAIN_DATASET_PATH = os.path.join(ROOT_DIR, "dataset")

# Dataset directories
LOW_DIR = "singlecoil_train_3D_images_48x/low/"
HIGH_DIR = "singlecoil_train_3D_images_48x/high/"

LOW_DIR_PATH = os.path.join(TRAIN_DATASET_PATH, LOW_DIR)
HIGH_DIR_PATH = os.path.join(TRAIN_DATASET_PATH, HIGH_DIR)
LOW_DIR_PATH, HIGH_DIR_PATH

('H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/low/',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/high/')

In [4]:
w, h = 128, 128
def rs_img(img):
    '''W and H is 128 now
    '''
    print(img.shape[-1])
    print(type(img.shape[-1]))
    flatten = [cv2.resize(img[:,:,i], (w, h), interpolation=cv2.INTER_CUBIC) for i in range(img.shape[-1])]
    img = np.array(np.dstack(flatten)) 
    return img


def change_depth(img):

    img_start = img[:,:,:4]
    
    mid = int(img.shape[-1]/2)
    img_middle = img[:,:,mid-10:mid+10]
    
    img_end = img[:,:,-4:]
    img = np.concatenate((img_start, img_middle, img_end), axis=2)
    return img


def show_slices(data, slice_nums, cmap=None):
    data = np.moveaxis(data, 2, 0)
    fig = plt.figure(figsize=(16, 16))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)        

In [5]:
len(os.listdir(LOW_DIR_PATH)), len(os.listdir(HIGH_DIR_PATH))

(972, 972)

In [6]:
low_full_paths =glob.glob(os.path.join(LOW_DIR_PATH,'*.npy'))
high_full_paths = glob.glob(os.path.join(HIGH_DIR_PATH,'*.npy'))

len(low_full_paths), len(high_full_paths)

(972, 972)

In [7]:
low_full_paths[:5]

['H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/low\\l1 (1).npy',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/low\\l1 (10).npy',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/low\\l1 (100).npy',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/low\\l1 (101).npy',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/low\\l1 (102).npy']

In [8]:
high_full_paths[:5]

['H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/high\\h (1).npy',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/high\\h (10).npy',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/high\\h (100).npy',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/high\\h (101).npy',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_3D_images_48x/high\\h (102).npy']

In [9]:
# Make folder for low and high res

LOW_DIR_2D = "singlecoil_train_2D_images/low/"
HIGH_DIR_2D = "singlecoil_train_2D_images/high/"

S.create_directory("{}/{}".format(TRAIN_DATASET_PATH, LOW_DIR_2D))
S.create_directory("{}/{}".format(TRAIN_DATASET_PATH, HIGH_DIR_2D))

LOW_DIR_2D,HIGH_DIR_2D

('singlecoil_train_2D_images/low/', 'singlecoil_train_2D_images/high/')

In [10]:
LOW_DIR_2D_PATH = os.path.join(TRAIN_DATASET_PATH, LOW_DIR_2D)
HIGH_DIR_2D_PATH = os.path.join(TRAIN_DATASET_PATH, HIGH_DIR_2D)
LOW_DIR_2D_PATH, HIGH_DIR_2D_PATH

('H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_2D_images/low/',
 'H:\\fastMRI-Reconstruction\\dataset\\singlecoil_train_2D_images/high/')

## Low MRI 3D to 2D image slices

In [11]:
c=0
total_slices = 0

x_train = []

for l in tqdm(low_full_paths[:]):
    img_3d = np.load("{}".format(l))
    img_3d = np.moveaxis(img_3d, 0, 2)
    slices = img_3d.shape[-1]
    total_slices+=slices
    
    # iterate over each slice and save
    for i in range(slices):
        img = img_3d[:,:,i]
        
        #x_train.append(img)
        
        #img = H.normalize(img) # Normalize image
        #cv2.imwrite("{}/{}_{}.png".format(LOW_DIR_2D_PATH, c, i), img * 255)
        np.save("{}/{}_{}.npy".format(LOW_DIR_2D_PATH, c, i), img)
        img = None
        
    img_3d = None
    c+=1

#x_train = np.array(x_train)
#print(x_train.shape)
print("Total slices: {}".format(total_slices))

100%|████████████████████████████████████████████████████████████████████████████████| 972/972 [09:30<00:00,  1.70it/s]

Total slices: 34707


# HIGH MRI VOLUMES

In [12]:
c=0
total_slices = 0

y_train = []

for l in tqdm(high_full_paths[:]):
    img_3d = np.load("{}".format(l))
    img_3d = np.moveaxis(img_3d, 0, 2)
    slices = img_3d.shape[-1]
    total_slices+=slices
    
    # iterate over each slice and save
    for i in range(slices):
        img = img_3d[:,:,i]
        
        
        #y_train.append(img)
        
        
        #img = H.normalize(img) # Normalize image
        #cv2.imwrite("{}/{}_{}.png".format(HIGH_DIR_2D_PATH, c, i), img * 255)
        np.save("{}/{}_{}.npy".format(HIGH_DIR_2D_PATH, c, i), img)
        img = None
        
    img_3d = None
    c+=1
    
#y_train = np.array(y_train)
#print(y_train.shape)
print("Total slices: {}".format(total_slices))

100%|████████████████████████████████████████████████████████████████████████████████| 972/972 [08:51<00:00,  1.83it/s]

Total slices: 34707
